<a href="https://colab.research.google.com/github/JonasSouza871/fruit_classifier_cnn/blob/main/fruit_classifier_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.  Treinamento da rede

-----------------

## 1.1 Carregamento dos dados do dataset

-----

In [1]:
!pip install kagglehub tensorflow pillow numpy matplotlib scikit-learn seaborn #dependências

- Importar biblioteca

In [2]:
import kagglehub
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import json

print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.19.0


In [3]:
# Download do dataset
path = kagglehub.dataset_download("moltean/fruits")
train_dir = None #diretorio de treino
test_dir = None #diretorio de teste

#Como o dataset está separado em imagens de treino e teste por pastas diferentes, temos que procurar os diretorios de treino (Train_dir) e teste(test_dir)
for root, dirs, files in os.walk(path):
    if '100x100' in root.lower():
        for d in dirs:
            full_path = os.path.join(root, d)
            if 'train' in d.lower() and train_dir is None:
                try:
                    if len([c for c in os.listdir(full_path) if os.path.isdir(os.path.join(full_path, c))]) > 10:
                        train_dir = full_path
                except:
                    pass
            if 'test' in d.lower() and test_dir is None:
                try:
                    if len([c for c in os.listdir(full_path) if os.path.isdir(os.path.join(full_path, c))]) > 10:
                        test_dir = full_path
                except:
                    pass
    if train_dir and test_dir:
        break

print(f"Diretório de Treino /validaçãop: {train_dir}")
print(f"Diretório de teste: {test_dir}")

100%|██████████| 4.61G/4.61G [01:58<00:00, 41.8MB/s]

Extracting files...


Diretório de Treino: /root/.cache/kagglehub/datasets/moltean/fruits/versions/63/fruits-360_100x100/fruits-360/Training
Diretório de Teste: /root/.cache/kagglehub/datasets/moltean/fruits/versions/63/fruits-360_100x100/fruits-360/Test


In [36]:
classes_finais = [ #classes selecionadas para o modelo treinar
    'Apple Golden 1',
    'Apple Red 1',
    'Strawberry 1',
    'Peach 1',
    'Pineapple 1',
    'Orange 1',
    'Lemon 1',
    'Mango 1',
    'Banana 1',
    'Pear 1'
]
print("CLASSES SELECIONADAS:")
val_split = 0.2

for i, classe in enumerate(classes_finais, 1): #para todas as classes
    train_path = os.path.join(train_dir, classe) #verifica as imagens de treino de cada classe no repositorio de treino
    test_path = os.path.join(test_dir, classe) #verifica as imagens de teste de cada classe no repositorio de teste
    total_train = len([f for f in os.listdir(train_path) if f.lower().endswith(('.jpg', '.png'))]) #total de imagens no train_dir
    train_imgs = int(total_train * (1 - val_split)) #80% para treino
    val_imgs = total_train - train_imgs #20% para validação
    test_imgs = len([f for f in os.listdir(test_path) if f.lower().endswith(('.jpg', '.png'))]) #verifica a quantidade de imagem de teste para cada classe
    print(f"{i:2d}. {classe:30s} | Treino: {train_imgs:4d} | Validação: {val_imgs:4d} | Teste: {test_imgs:4d}")

CLASSES SELECIONADAS:
 1. Apple Golden 1                 | Treino:  384 | Validação:   96 | Teste:  160
 2. Apple Red 1                    | Treino:  393 | Validação:   99 | Teste:  164
 3. Strawberry 1                   | Treino:  393 | Validação:   99 | Teste:  164
 4. Peach 1                        | Treino:  393 | Validação:   99 | Teste:  164
 5. Pineapple 1                    | Treino:  392 | Validação:   98 | Teste:  166
 6. Orange 1                       | Treino:  383 | Validação:   96 | Teste:  160
 7. Lemon 1                        | Treino:  393 | Validação:   99 | Teste:  164
 8. Mango 1                        | Treino:  392 | Validação:   98 | Teste:  166
 9. Banana 1                       | Treino:  392 | Validação:   98 | Teste:  166
10. Pear 1                         | Treino:  393 | Validação:   99 | Teste:  164


## 1.2. Configuração do modelo de treino

-------

In [37]:
IMG_SIZE = 100 #tamanho da imagem
BATCH_SIZE = 32
EPOCHS = 100  #quantidade de epocas
LEARNING_RATE = 0.0001 #taxa de aprendizado

# Data augmentation para treino/validação
#preparação da imagem para treinamento
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2 #validação considerando 20% dos dados totais
)
# Data generator para teste (apenas rescale)
test_datagen = ImageDataGenerator(rescale=1./255)

In [39]:
# Gerador de treino
train_generator = datagen.flow_from_directory(
    train_dir, #pega as iamgens no diretorio de trieno
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training', # subsset de trieno
    classes=classes_finais, #com base nas classes definidos acima
    shuffle=True,
    seed=42 #garantir reprodutibilidade
)

# Gerador de validação
validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=classes_finais,
    shuffle=False,
    seed=42
)

# Gerador de teste
test_generator = test_datagen.flow_from_directory(
    test_dir, #usa o diretorio separado para os dados de test do proprio dataset
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes_finais,
    shuffle=False
)

# Informações sobre a divisão dos dados
num_classes = len(classes_finais)
print("DIVISÃO DOS DADOS:")
print(f"Treino:     {train_generator.samples:,} imagens ({train_generator.samples/(train_generator.samples+validation_generator.samples)*100:.1f}%)") #separa os dadops de cada classe que serão para treino (definido para 80%)
print(f"Validação:  {validation_generator.samples:,} imagens ({validation_generator.samples/(train_generator.samples+validation_generator.samples)*100:.1f}%)") #separa os dados de cada classe que serão para validação (definido para)
print(f"Classes:    {num_classes}")

# Salvar labels
labels = {v: k for k, v in train_generator.class_indices.items()} #labels atraves do treino
with open('labels.json', 'w') as f:
    json.dump(labels, f, indent=2)

Found 3914 images belonging to 10 classes.
Found 975 images belonging to 10 classes.
Found 1638 images belonging to 10 classes.
DIVISÃO DOS DADOS:
Treino:     3,914 imagens (80.1%)
Validação:  975 imagens (19.9%)
Classes:    10


## 1.3. Arquitetura da CNN

-----

In [40]:
#Construção da arquitetura CNN
model = keras.Sequential([
    #camadas convolucionais 2d com atiavação via relu e maxpooling
    layers.Conv2D(8, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2), #dropout para melhor a generalização do modelo
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(32, activation='relu'), #camada simples ao final
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax') #camada final aplicando ativação com SOFTMAX
])

# Compilação do modelo
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE), #otimizador ADAM
    #metricas que irão ser avlaiadas
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 98, 98, 8)      │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 49, 49, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 47, 47, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 23, 23, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 23, 23, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 21, 21, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 10, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 10, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │       102,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 108,794 (424.98 KB)

 Trainable params: 108,794 (424.98 KB)

 Non-trainable params: 0 (0.00 B)

## 1.4. Configuração de Callbacks e Treinamento

-----

In [ ]:
# Callbacks para controle do treinamento
early_stop = EarlyStopping( #parada precose se o modelo não evoluir por 10 epocas
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)
reduce_lr = ReduceLROnPlateau( #reduz automaticamente a taxa de aprendizado (learning rate) quando o modelo para de melhorar na métrica que você está monitorando
    monitor='val_loss',#metrica a ser avaliada
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

callbacks = [early_stop, reduce_lr] #parametros de controle
history = model.fit( #ativa o treinamento do modelo
    train_generator, #dados de treino
    epochs=EPOCHS, #epocas que serao utilizadas
    validation_data=validation_generator, #dados de validação
    callbacks=callbacks #callbacks de controle q irão aocmpanhar o modelo
)

Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


123/123 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.1437 - loss: 2.2772 - val_accuracy: 0.2708 - val_loss: 2.1426 - learning_rate: 1.0000e-04
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 15s 126ms/step - accuracy: 0.2871 - loss: 1.9540 - val_accuracy: 0.5672 - val_loss: 1.4557 - learning_rate: 1.0000e-04
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 17s 140ms/step - accuracy: 0.4404 - loss: 1.4451 - val_accuracy: 0.7867 - val_loss: 0.9176 - learning_rate: 1.0000e-04
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 18s 146ms/step - accuracy: 0.5737 - loss: 1.1018 - val_accuracy: 0.8256 - val_loss: 0.6832 - learning_rate: 1.0000e-04
Epoch 5/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 16s 134ms/step - accuracy: 0.6497 - loss: 0.8988 - val_accuracy: 0.8431 - val_loss: 0.5453 - learning_rate: 1.0000e-04
Epoch 6/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 26s 211ms/step - accuracy: 0.7059 - loss: 0.7635 - val_accuracy: 0.8821 - val_loss: 0.4365 - learning_rate: 1.0000e-04
Epoch 7/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 17s 137ms/s

# 1.5. Plotagens e visualização das métricas de treinamento

--------------

In [ ]:
# Plotagem das curvas de aprendizado
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia')
plt.legend()
plt.grid(True)
plt.ylim(0, 1.05)
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Avaliação do modelo no conjunto de teste
test_loss, test_accuracy = model.evaluate(test_generator) #aaplica os resultados dos dados de treino (validação e treino) no dataset de teste
print("RESULTADO NO CONJUNTO DE TESTE:")
print(f"Loss:     {test_loss:.4f}")
print(f"Acurácia: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)") #acuracia em %

In [ ]:
# Gerando predições para análise detalhada
test_generator.reset()
y_pred = [] #classe de previsao do modelo
y_true = [] #classe verdadeira apartir das labels

for i in range(len(test_generator)): #para todas as imagens do dataset de teste
    x_batch, y_batch = next(test_generator) #x_batch → um lote de imagens (ex: 32 imagens 150x150),  y_batch → os rótulos one-hot correspondentes a essas imagens
    predictions = model.predict(x_batch, verbose=0) #O modelo faz as previsões para todas as imagens do batch.
    y_pred.extend(np.argmax(predictions, axis=1)) #pega a classe de maior probabilidade para cada imagem.
    y_true.extend(np.argmax(y_batch, axis=1)) #converte todos one-hot da labels em classe verdadeira (O que de fato é)

#cria array
y_pred = np.array(y_pred)
y_true = np.array(y_true)

# Matriz de confusão
cm = confusion_matrix(y_true, y_pred) #vai compara o que foi previsto pelo modelo apartir da predição de cada imagem e o que é verdade apartir das labels reais

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes_finais,
            yticklabels=classes_finais,
            cbar_kws={'label': 'Quantidade'})
plt.title('Matriz de Confusão', fontsize=16, pad=20)
plt.ylabel('Classe Verdadeira', fontsize=12)
plt.xlabel('Classe Prevista', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Relatório de classificação completo
print("RELATÓRIO DE CLASSIFICAÇÃO:")
print(classification_report(y_true, y_pred, target_names=classes_finais, digits=4)) #f1 score, recall, acuracia ..

print("\nACURÁCIA POR CLASSE:")
for i, cls in enumerate(classes_finais): # para cada classe irá fazer uma analise completa dos resultados aparitr da acuracia
    cls_mask = y_true == i
    cls_acc = np.mean(y_pred[cls_mask] == y_true[cls_mask])
    total = np.sum(cls_mask)
    corretos = np.sum(y_pred[cls_mask] == y_true[cls_mask])
    print(f"{cls:30s} | {corretos:3d}/{total:3d} | {cls_acc*100:5.2f}%")

## 1.6. Salvando o modelo

------

In [ ]:
#Salvamento do modelo Keras
model.save('modelo_fruits.keras') #salva o modelo de treinamento no modelo Keras
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()# Conversão e salvamento do modelo TFLite

with open('modelo_fruits.tflite', 'wb') as f:
    f.write(tflite_model)

#Comparação de tamanhos de cada modelo keras e tflite
keras_size = os.path.getsize('modelo_fruits.keras') / 1024 / 1024
tflite_size = os.path.getsize('modelo_fruits.tflite') / 1024 / 1024
print(f"\nTamanho Keras:  {keras_size:.2f} MB")
print(f"Tamanho TFLite: {tflite_size:.2f} MB")
print(f"Redução:        {(1 - tflite_size/keras_size)*100:.1f}%")

#Download dos arquivos (apenas funciona no Colab)
try:
    from google.colab import files
    files.download('modelo_fruits.keras')
    files.download('modelo_fruits.tflite')
    files.download('labels.json')
    print("Download concluído!")
except:
    print("erro")